In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from tensorflow.keras import Sequential,layers,losses,utils,Input
from tensorflow.keras.layers import Dense,LSTM,Dropout,concatenate,Flatten, Conv1D, MaxPooling1D,Activation,RepeatVector,TimeDistributed
import tensorflow as tf
# from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Bidirectional,GRU,Lambda,Dot,Concatenate
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
from keras.layers import Bidirectional,LSTM,GRU,Lambda,Dot,Concatenate

import matplotlib.pyplot as plt
import matplotlib

from sklearn.decomposition import PCA
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras import optimizers
from keras.models import load_model

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
data=pd.read_csv('hunhe.csv')

In [ ]:
data

In [ ]:
data.columns

In [6]:
col=[ 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2',
       'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN',  'WSPM','wd','shunyiPM2.5']
datacol=data[col]

In [ ]:
datacol.isnull().sum()

In [8]:
# 直接删除缺失值
datashan=datacol.dropna()

In [ ]:
datashan.isnull().sum()

In [ ]:
datashan.shape

In [ ]:
datashan.describe()

In [ ]:
# 拼接时间为一列，绘制一年时间降水分布图
# datag['时间(年月日时)']=datag['年(年)'].map(str)+''+datag['月(月)'].map(str)+''+datag['日(日)'].map(str)+''+datag['时(时)'].map(str)
datashan['datatime']=datashan['year'].map(str)+'-'+datashan['month'].map(str)+'-'+datashan['day'].map(str)+'-'+datashan['hour'].map(str)

In [ ]:
datashan.head(8)

In [ ]:
# datag=data.copy()
datashan['datatime']=pd.to_datetime(datashan['datatime'],format='%Y-%m-%d-%H',errors='coerce')
datashan.index=datashan['datatime']

In [ ]:
datashan.isnull().sum()

In [ ]:
# 给风向编码
datashan['wd'].unique()

In [ ]:
scale_mapper={'E':1, 'ENE':2, 'NNE':3, 'N':4, 'NNW':5, 'NW':6, 'NE':7, 'SW':8, 'SSW':9, 'WSW':10,
       'ESE':11, 'SE':12, 'S':13, 'WNW':14, 'SSE':15, 'W':16}
datashan['wd']=datashan['wd'].replace(scale_mapper)

In [18]:
chongcol=[ 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM','wd','shunyiPM2.5']
datag=datashan[chongcol]

In [19]:
datal=datashan[chongcol]

In [ ]:
datal

In [ ]:
datal.max()

In [ ]:
datag

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [ ]:
# 归一化
scaler = MinMaxScaler(feature_range=(0, 1))  # minmaxscaler 方法，用来做归一化
sel_col = datag.columns
# print(sel_col)
for col in sel_col:
    datag[col] = scaler.fit_transform(datag[col].values.reshape(-1, 1))  # reshape成为一列
print("Load dataset LEN: ", datag.shape[0])
print(sel_col)


In [ ]:
# 归一化后的数据
datag

In [62]:
# datag.corr()

In [ ]:
X = []
Y = []
columns_name = list(datag.columns)
y_index = columns_name.index('PM2.5')
tmp = np.array(datag, dtype=np.float32)
print("The shape of tmp is:")
print(tmp.shape)
pca = PCA(n_components=0.97)# 保证降维后的数据保持90%的信息
tmp2 = pca.fit_transform(tmp[:,1:])
print("The shape of tmp2 is:")
print(tmp2.shape)  # PCA降维y_index要改，或者先提取出来
print("input dim is:")
print(tmp2.shape[1])

In [129]:
# datax_trans=tmp2
# datay_trans=tmp[:,0]

In [130]:
# datax_trans

In [131]:
# datax_trans.shape

In [132]:
# datay_trans

In [133]:
# datay_trans.shape

In [64]:
seqX=120
seqY=1
day_delay=1
gap=1
X1=[]
for i in range(tmp2.shape[0] - (seqX+seqY+day_delay)*gap):
        # seqX, seqX+day_delay, seqX+day_delay+seqY
        # format: i+x*gap
        X.append(tmp2[i:(i + seqX*gap):gap, :])  # tmp2是PCA降维后的特征，如果是tmp那就是原始特征
        X1.append(tmp2[(i+seqX*gap): (i+(seqX+day_delay)*gap): gap,:])
#         Y.append(tmp[(i+(seqX+day_delay)*gap): (i+(seqX+day_delay+seqY)*gap): gap, y_index])
        Y.append(tmp[(i+seqX*gap): (i+(seqX+day_delay)*gap): gap, y_index])
X = np.array(X)
Y = np.array(Y)
X1=np.array(X1)

划分数据集

In [65]:
# x_train, x_test,  y_train, y_test = train_test_split(datax_trans, datay_trans, test_size = 0.2, random_state = 666,shuffle=False)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X1.shape

In [69]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)
p = np.random.permutation(range(len(X)))
X,Y,X1 = X[p],Y[p],X1[p]

In [71]:
shendu_trainx, shendu_trainy = X[:int(0.9 * 31792)], Y[:int(0.9 * 31792)]  
jiqi_trainx, jiqi_trainy = X1[:int(0.9 * 31792)], Y[:int(0.9 * 31792)]  

In [77]:
shendu_testx, shendu_testy = X[int(0.9 * 31792):], Y[int(0.9 * 31792):] 
jiqi_testx, jiqi_testy = X1[int(0.9 * 31792):], Y[int(0.9 * 31792):] 

In [ ]:
shendu_trainx.shape

In [ ]:
jiqi_trainx.shape

In [ ]:
shendu_testx.shape

In [ ]:
jiqi_testx.shape

In [ ]:
jiqi_trainy.shape

In [ ]:
jiqi_testy.shape

转换成机器学习可以用的数据

In [84]:
realjiqi_trainx=jiqi_trainx.reshape(28612,8)

In [ ]:
realjiqi_trainx

In [87]:
realjiqi_testx=jiqi_testx.reshape(3180,8)

In [ ]:
realjiqi_testx

In [ ]:
jiqi_testx

In [ ]:
realjiqi_testx.shape

保存数据

In [91]:
# 保存深度学习训练测试数据
# 将训练集存起来
nptrainx=np.reshape(shendu_trainx,(-1,8))
pdtrainx1=np.reshape(nptrainx,(-1,120,8))

In [ ]:
pdtrainx1.shape

In [93]:
pdshendutrainx=pd.DataFrame(nptrainx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdshendutrainx.to_csv('shendux_train.csv',index=False)

In [94]:
nptestx=np.reshape(shendu_testx,(-1,8))

In [95]:
pdshendux_test=pd.DataFrame(nptestx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdshendux_test.to_csv('shendux_test.csv',index=False)

In [96]:
pdshenduy_train=pd.DataFrame(shendu_trainy,columns=['y1'])
pdshenduy_train.to_csv('shenduy_train.csv',index=False)

In [97]:
pdshenduy_test=pd.DataFrame(shendu_testy,columns=['y1'])
pdshenduy_test.to_csv('shenduy_test.csv',index=False)

In [98]:
# 保存机器学习训练测试集
pdjiqitrainx=pd.DataFrame(realjiqi_trainx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdjiqitrainx.to_csv('jiqix_train.csv',index=False)

In [99]:
pdjiqitestx=pd.DataFrame(realjiqi_testx,columns=['t1','t2','t3','t4','t5','t6','t7','t8'])
pdjiqitestx.to_csv('jiqix_test.csv',index=False)

In [100]:
pdjiqiy_train=pd.DataFrame(jiqi_trainy,columns=['y1'])
pdjiqiy_train.to_csv('jiqiy_train.csv',index=False)

In [101]:
pdjiqiy_test=pd.DataFrame(jiqi_testy,columns=['y1'])
pdjiqiy_test.to_csv('jiqiy_test.csv',index=False)

In [ ]:
jiqi_testy

In [ ]:
shendu_testy

读取深度学习数据

In [192]:
# 读取训练集
dushendux_train=pd.read_csv('shendux_train.csv')
dushendux_train1=np.array(dushendux_train)
dushendux_train2=np.reshape(dushendux_train1,(-1,120,7))

dushenduy_train=pd.read_csv('shenduy_train.csv')
dushenduy_train2=np.array(dushenduy_train)
# 读取测试集
dushendux_test=pd.read_csv('shendux_test.csv')
dushendux_test1=np.array(dushendux_test)
dushendux_test2=np.reshape(dushendux_test1,(-1,120,7))

dushenduy_test=pd.read_csv('shenduy_test.csv')
dushenduy_test2=np.array(dushenduy_test)

In [ ]:
dushendux_train2

In [ ]:
dushendux_train2.shape

In [ ]:
dushendux_test2.shape

读取数据机器学习

In [3]:
# 读取训练集
dujiqix_train=pd.read_csv('jiqix_train.csv')
dujiqiy_train=pd.read_csv('jiqiy_train.csv')
# 读取测试集
dujiqix_test=pd.read_csv('jiqix_test.csv')
dujiqiy_test=pd.read_csv('jiqiy_test.csv')

In [ ]:
dujiqix_train

In [ ]:
dujiqix_train

In [ ]:
dujiqix_test

In [ ]:
dujiqiy_train

In [ ]:
jiqi_trainy

In [ ]:
dux_test

In [ ]:
duy_train

In [ ]:
duy_test

方法一：线性预测

In [202]:
from sklearn.metrics import mean_absolute_error

In [89]:
from sklearn.linear_model import LinearRegression,RidgeCV,SGDRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score   #均方误差

In [344]:
# # # # 1.线性回归----此处使用基模型，参数均使用默认的，用于后面调参对比
# Lr=LinearRegression(normalize=False)
# Lr.fit(dujiqix_train,dujiqiy_train)
# xy_predict=Lr.predict(dujiqix_test)
# xy_modify = xy_predict*(xy_predict>=0)
# r2=r2_score(dujiqiy_test,xy_modify)
# # Mae=mean_squared_error(y_test,xy_modify)
# r2

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [ ]:
maxmin

In [345]:
# xpreds = np.array(xy_modify)  # 转换为numpy形式
# xlabels = np.array(dujiqiy_test)
# xpreds= xpreds.reshape((-1,1))
# xfuture_len= xpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     xlabels[:, k] = xlabels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(xfuture_len):
#     xpreds[:, k] = xpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [346]:
# np.sqrt(mean_squared_error(xlabels,xpreds))

In [347]:
# np.sqrt(mean_absolute_error(xlabels,xpreds))

In [343]:
# r2_score(xlabels,xpreds)

决策树

In [341]:
# tree_model=DecisionTreeRegressor()
# tree_model.fit(dux_train,duy_train)
# treey_pre=tree_model.predict(dux_test)
# ty_modify = treey_pre*(treey_pre>=0)
# jr=r2_score(duy_test,ty_modify)
# # treemae2=mean_squared_error(y_test,ty_modify2)
# # print(jr2,treemae2)
# jr

In [342]:
#  # 数据逆归一化
# maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
# print(maxmin)

In [349]:
# preds = np.array(ty_modify)  # 转换为numpy形式
# labels1 = np.array(duy_test)
# preds= preds.reshape((-1,1))
# future_len= preds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(future_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
# for k in range(future_len):
#     preds[:, k] = preds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
preds

In [ ]:
labels1

In [101]:
treemae=mean_squared_error(labels1,preds)

In [ ]:
treemae

In [ ]:
np.sqrt(mean_squared_error(labels1,preds))

In [ ]:
r2_score(labels1,preds)

随机森林

In [105]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rd=RandomForestRegressor(max_depth=11,max_features=6,min_samples_split=35,n_estimators=96,min_samples_leaf=9)     #基模型
rd.fit(dujiqix_train,dujiqiy_train)
suiy_pres=rd.predict(dujiqix_test)
yrand_modify = suiy_pres*(suiy_pres>=0)
suijir=r2_score(dujiqiy_test,yrand_modify)
suijir

In [49]:
label = np.array(dujiqiy_test)
for k in range(1):
#     print(k)
    label[:, k] = label[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(dujiqiy_test)

In [369]:
spreds = np.array(yrand_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
spreds= spreds.reshape((-1,1))
sfuture_len= spreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(sfuture_len):
    spreds[:, k] = spreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
spreds

In [ ]:
np.sqrt(mean_squared_error(label,spreds))

In [ ]:
mean_absolute_error(label,spreds)

保存预测结果

In [208]:
# spreds1=spreds.reshape(-1,1)

In [209]:
spreds1=spreds.reshape(-1,1)
baocunspreds=pd.DataFrame(spreds1,columns=['y_pred'])
baocunspreds.to_csv('随机森林预测结果.csv',index=False)

In [ ]:
a=pd.read_csv('随机森林预测结果.csv')
a

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('随机森林模型',fontsize=25)
ax.plot(labels1, "r", label="real", alpha=0.7,ms=7)
ax.plot(a, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文随机森林拟合图.jpg")

Adaboost算法

In [372]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=11, min_samples_split=35, min_samples_leaf=35),
                            n_estimators=50,learning_rate=0.1,loss='linear')
Ada_model.fit(dujiqix_train,dujiqiy_train)
aday_pres=Ada_model.predict(dujiqix_test)
aday_modify = aday_pres*(aday_pres>=0)
adar=r2_score(dujiqiy_test,aday_modify)
adar

In [ ]:
# 也可以对选择的弱回归器进行参数选择，方式为base_estimator__若回归器参数名
Ada_model1=AdaBoostRegressor(DecisionTreeRegressor(max_depth=11, min_samples_split=10, min_samples_leaf=30),
                            n_estimators=50,learning_rate=0.1,loss='square')
Ada_model1.fit(dujiqix_train,dujiqiy_train)
aday_pres1=Ada_model.predict(dujiqix_test)
aday_modify1 = aday_pres1*(aday_pres1>=0)
adar1=r2_score(dujiqiy_test,aday_modify1)
adar1

In [376]:
apreds = np.array(aday_modify)  # 转换为numpy形式
# labels1 = np.array(dujiqiy_test)
apreds= apreds.reshape((-1,1))
afuture_len= apreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(afuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(afuture_len):
    apreds[:, k] = apreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,apreds))

In [ ]:
r2_score(label,apreds)

In [ ]:
mean_absolute_error(label,apreds)

保存预测结果

In [381]:
apreds1=apreds.reshape(-1,1)
baocunapreds=pd.DataFrame(apreds1,columns=['y_pred'])
baocunapreds.to_csv('AdaBooost预测结果.csv',index=False)

In [ ]:
b=pd.read_csv('AdaBooost预测结果.csv')
b

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
ax.plot(labels1, "r", label="real", alpha=0.7,ms=7)
ax.plot(b, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文AdaBoost拟合图.jpg")

GBRT

In [384]:
from sklearn.ensemble import GradientBoostingRegressor

In [385]:
# Gbrt_model= GradientBoostingRegressor(n_estimators=50,max_depth=7,min_samples_split=10,learning_rate=0.1,min_samples_leaf=5)#这里使用50个决策树
# Gbrt_model.fit(dujiqix_train,dujiqiy_train)
# gbrty_pres=Gbrt_model.predict(dujiqix_test)
# gbrty_modify= gbrty_pres*(gbrty_pres>=0)
# gbrtr=r2_score(dujiqiy_test,gbrty_modify)
# gbrtr

In [ ]:
Gbrt_model= GradientBoostingRegressor()#这里使用50个决策树
Gbrt_model.fit(dujiqix_train,dujiqiy_train)
gbrty_pres=Gbrt_model.predict(dujiqix_test)
gbrty_modify = gbrty_pres*(gbrty_pres>=0)
gbrtr=r2_score(dujiqiy_test,gbrty_modify)
gbrtr

In [388]:
gpreds = np.array(gbrty_modify)  # 转换为numpy形式
# labels2 = np.array(dujiqiy_test)
gpreds= gpreds.reshape((-1,1))
gfuture_len= gpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(gfuture_len):
#     labels2[:, k] = labels2[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(gfuture_len):
    gpreds[:, k] = gpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
gpreds.shape[1] 

In [ ]:
np.sqrt(mean_squared_error(label,gpreds))

In [ ]:
mean_absolute_error(label,gpreds)

In [ ]:
np.sum(np.absolute(labels1 - gpreds)) / len(dujiqiy_test)

In [ ]:
label

保存预测结果

In [310]:
gpreds=gpreds.reshape(-1,1)
baocungpreds=pd.DataFrame(gpreds,columns=['y_pred'])
baocungpreds.to_csv('GBRT预测结果.csv',index=False)

In [ ]:
c=pd.read_csv('GBRT预测结果.csv')
c

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('AdaBoost模型',fontsize=25)
ax.plot(labels, "r", label="real", alpha=0.7,ms=7)
ax.plot(c, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文GBRT拟合图.jpg")

xgboost

In [64]:
from xgboost import XGBRegressor

In [ ]:
other_params = {'learning_rate': 0.1, 'n_estimators': 75, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,}
xgb_model1=XGBRegressor(**other_params)
xgb_model1.fit(dujiqix_train,dujiqiy_train)
xgby1_pred=xgb_model1.predict(dujiqix_test)
xgby1_modify = xgby1_pred*(xgby1_pred>=0)
xgbr1=r2_score(dujiqiy_test,xgby1_modify)
xgbr1

In [ ]:
other_params1 = {'learning_rate': 0.1, 'n_estimators': 70, 'max_depth': 4, 'min_child_weight': 5, 
                     'colsample_bytree': 0.7, 'gamma': 0.01,
                 }
xgb_model=XGBRegressor(**other_params1)
xgb_model.fit(dujiqix_train,dujiqiy_train)
xgby_pred=xgb_model.predict(dujiqix_test)
xgby_modify = xgby_pred*(xgby_pred>=0)
xgbr=r2_score(dujiqiy_test,xgby_modify)
xgbr

In [66]:
xgbpreds = np.array(xgby1_modify)  # 转换为numpy形式
# labels3 = np.array(dujiqiy_test)
xgbpreds= xgbpreds.reshape((-1,1))
xfuture_len= xgbpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(xfuture_len):
#     labels3[:, k] = labels3[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for j in range(xfuture_len):
    xgbpreds[:, j] = xgbpreds[:, j] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.array(xgby1_modify).reshape(-1,1)

In [ ]:
xgbpreds

In [ ]:
label

In [ ]:
np.sqrt(mean_squared_error(labels,xgbpreds))

In [ ]:
mean_absolute_error(labels,xgbpreds)

In [ ]:
r2_score(labels,xgbpreds)

保存预测结果

In [448]:
xgbpreds=xgbpreds.reshape(-1,1)
baocunxgbpreds=pd.DataFrame(xgbpreds,columns=['y_pred'])
baocunxgbpreds.to_csv('XGBoost预测结果.csv',index=False)

In [ ]:
d=pd.read_csv('XGBoost预测结果.csv')
d

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(d, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文XGBoost拟合图.jpg")

lightGBM

In [451]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [ ]:
tmodel_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=36,
                              learning_rate=0.01, n_estimators=430, max_depth=8, 
                              metric='rmse', min_child_samples=4,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb.fit(dujiqix_train,dujiqiy_train)
lgby=tmodel_lgb.predict(dujiqix_test)
lgby_modify=lgby*(lgby>=0)
lgbr=r2_score(dujiqiy_test,lgby_modify)
lgbr

In [ ]:
tmodel_lgb1 = lgb.LGBMRegressor(objective='regression',num_leaves=30,
                              learning_rate=0.01, n_estimators=500, max_depth=8, 
                              metric='rmse', min_child_samples=5,
                            )#reg_alpha=0.5,reg_lambda=0.5,feature_fraction=0.6,bagging_fraction=0.4,
tmodel_lgb1.fit(dujiqix_train,dujiqiy_train)
lgby1=tmodel_lgb1.predict(dujiqix_test)
lgby_modify1=lgby1*(lgby1>=0)
lgbr1=r2_score(dujiqiy_test,lgby_modify1)
lgbr1

In [489]:
lpreds1 = np.array(lgby_modify1)  # 转换为numpy形式
# llabel = np.array(dujiqiy_test)
lpreds1= lpreds1.reshape((-1,1))
lfuture_len= lpreds1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lfuture_len):
#     llabel[:, k] = llabel[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lfuture_len):
    lpreds1[:, k] = lpreds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lpreds1))

In [ ]:
mean_absolute_error(label,lpreds1)

In [ ]:
mean_squared_error(llabel,lpreds)

保存预测结果

In [493]:
lpreds1=lpreds1.reshape(-1,1)
baocunlpreds=pd.DataFrame(lpreds1,columns=['y_pred'])
baocunlpreds.to_csv('LightGBM预测结果.csv',index=False)

In [ ]:
e=pd.read_csv('LightGBM预测结果.csv')
e

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('XGBoost模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(e, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文LightGBM拟合图.jpg")

SVR

In [468]:
from sklearn.svm import SVR

In [ ]:
svr_rbf = SVR(kernel='linear', C=10, gamma=1)
svr_rbf.fit(dujiqix_train,dujiqiy_train)
svry_pred= svr_rbf.predict(dujiqix_test)
svry_modify= svry_pred*(svry_pred>=0)
svrr=r2_score(dujiqiy_test,svry_modify)
svrr

In [ ]:
svr_rbf2 = SVR(kernel='rbf', C=10, gamma=1)
svr_rbf2.fit(dujiqix_train,dujiqiy_train)
svry_pred2= svr_rbf2.predict(dujiqix_test)
svry_modify2 = svry_pred2*(svry_pred2>=0)
svrr2=r2_score(dujiqiy_test,svry_modify2)
svrr2

In [474]:
svrpreds = np.array(svry_modify2)  # 转换为numpy形式
# labels1 = np.array(y_test)
svrpreds= svrpreds.reshape((-1,1))
svrfuture_len= svrpreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(sfuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(svrfuture_len):
    svrpreds[:, k] = svrpreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,svrpreds))

In [ ]:
mean_absolute_error(label,svrpreds)

保存预测结果

In [477]:
svrpreds=svrpreds.reshape(-1,1)
baocunsvrpreds=pd.DataFrame(svrpreds,columns=['y_pred'])
baocunsvrpreds.to_csv('SVR预测结果.csv',index=False)

In [ ]:
f=pd.read_csv('SVR预测结果.csv')
f

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('SVR模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(f, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文SVR拟合图.jpg")

lasso回归

In [480]:
from sklearn.linear_model import LassoCV

In [ ]:
La=LassoCV()
La.fit(dujiqix_train,dujiqiy_train)
lay_predict=La.predict(dujiqix_test)
lay_modify = lay_predict*(lay_predict>=0)
rla=r2_score(dujiqiy_test,lay_modify)
rla

In [483]:
lapreds = np.array(lay_modify)  # 转换为numpy形式
# labels1 = np.array(y_test)
lapreds= lapreds.reshape((-1,1))
lafuture_len= lapreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
# for k in range(lafuture_len):
#     labels1[:, k] = labels1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化
for k in range(lafuture_len):
    lapreds[:, k] = lapreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
np.sqrt(mean_squared_error(label,lapreds))

In [ ]:
mean_absolute_error(label,lapreds)

保存预测结果

In [486]:
lapreds=lapreds.reshape(-1,1)
baocunlapreds=pd.DataFrame(lapreds,columns=['y_pred'])
baocunlapreds.to_csv('Lasso预测结果.csv',index=False)

In [ ]:
g=pd.read_csv('Lasso预测结果.csv')
g

In [ ]:
# LSTM拟合图
plt.figure(figsize=(25, 8))
ax = plt.subplot(111)
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Lasso模型',fontsize=25)
ax.plot(label, "r", label="real", alpha=0.7,ms=7)
ax.plot(g, "g--", label="pred", alpha=0.8,ms=7)
plt.xlabel('time',fontsize=20)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 20})
plt.legend(fontsize=20)
plt.savefig("论文Lasso拟合图.jpg")

# 深度学习算法

In [24]:
# 读取训练集
dushendux_train=pd.read_csv('shendux_train.csv')
dushendux_train1=np.array(dushendux_train)
dushendux_train2=np.reshape(dushendux_train1,(-1,120,8))

dushenduy_train=pd.read_csv('shenduy_train.csv')
dushenduy_train2=np.array(dushenduy_train)
# 读取测试集
dushendux_test=pd.read_csv('shendux_test.csv')
dushendux_test1=np.array(dushendux_test)
dushendux_test2=np.reshape(dushendux_test1,(-1,120,8))

dushenduy_test=pd.read_csv('shenduy_test.csv')
dushenduy_test2=np.array(dushenduy_test)

In [25]:
labels= np.array(dushenduy_test2)
for k in range(labels.shape[1]):
    labels[:, k] = labels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
dushendux_train2

In [ ]:
dushendux_train2.shape

In [ ]:
dushendux_test2.shape

In [ ]:
dushenduy_train2.shape

In [ ]:
dushenduy_test2.shape

In [31]:
# # # 构造批数据
def create_batch_dataset(x,y,train=True,buffer_size=1000,batch_size=128):#buffer_size=1000表示可以打乱窗口里面的数据
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))#数据封装，tensor类型
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

In [32]:
train_data_single=create_batch_dataset(dushendux_train2, dushenduy_train2,train=True)
val_data_single=create_batch_dataset(dushendux_test2,dushenduy_test2,train=False)

In [33]:
# # # 构造批数据
def create_batch_dataset64(x,y,train=True,buffer_size=1000,batch_size=64):#buffer_size=1000表示可以打乱窗口里面的数据
    batch_data=tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))#数据封装，tensor类型
    if train:
        return batch_data.cache().shuffle(buffer_size).batch(batch_size)
    else:
        return batch_data.batch(batch_size)

In [34]:
train_data_single64=create_batch_dataset64(dushendux_train2, dushenduy_train2,train=True)
val_data_single64=create_batch_dataset64(dushendux_test2,dushenduy_test2,train=False)

In [ ]:
train_data_single

In [ ]:
dushendux_train2.shape[-2:]

In [ ]:
dushendux_train2.shape

In [35]:
# 添加注意力机制
def attention_bilstm(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [36]:
# 添加注意力机制
def attention_bilstm64(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(64, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [37]:
# 添加注意力机制
def attention_bilstm32(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(32, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
bi_gru1=Bidirectional(LSTM(128,return_sequences=True))(model_input)

bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(bi_gru1)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru=attention_bilstm(bi_gru2)
# a=Dropout(0.2)(attbigru)
tt=Dense(1)(attbigru)
attbilstm_model=Model(model_input,tt)
attbilstm_model.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history = attbilstm_model.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1)

In [ ]:
# 预测
attbilstm_pres=attbilstm_model.predict(dushendux_test2,verbose=1)

In [ ]:
attbilstmy_modify = attbilstm_pres*(attbilstm_pres>=0)
attbilstmr=r2_score(dushenduy_test2,attbilstmy_modify)
attbilstmr

In [ ]:
保存模型

In [ ]:
attbilstm_model.save('论文saveattbiLSTM_model')

In [36]:
pd.DataFrame(attbilstm_history.history).to_csv('论文saveattbiLSTM_history.csv', index=False)

In [ ]:
a1=pd.read_csv('论文saveattbiLSTM_history.csv')
a1

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(a1['val_loss'],label='Bi-LSTM-Attention_loss_value')
plt.plot(a1['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文Bi-LSTMAttention损失曲线.jpg")
plt.show()

第二版

In [37]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [48]:
filepath = "weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num2_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a2=Dropout(0.2)(num2_bi_gru)
num2_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru2=attention_bilstm(num2_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt2=Dense(1)(attbigru2)
attbilstm_model=Model(model_input,tt2)
attbilstm_model.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history = attbilstm_model.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint)

In [ ]:
# 预测
attbilstm_pres1=attbilstm_model.predict(dushendux_test2,verbose=1)

In [ ]:
attbilstmy1_modify = attbilstm_pres1*(attbilstm_pres1>=0)
attbilstmr1=r2_score(dushenduy_test2,attbilstmy1_modify)
attbilstmr1

In [ ]:
attbilstm_model.save('论文1saveattbiLSTM_model')

In [53]:
pd.DataFrame(attbilstm_history.history).to_csv('论文1saveattbiLSTM_history.csv', index=False)

In [ ]:
a2=pd.read_csv('论文1saveattbiLSTM_history.csv')
a2

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(a2['val_loss'],label='Bi-LSTM-Attention_loss_value')
plt.plot(a2['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文1Bi-LSTMAttention损失曲线.jpg")
plt.show()

In [60]:
from keras.models import load_model
zuihaomodel=load_model("weights-improvement-736-0.00.hdf5")

In [ ]:
zui=zuihaomodel.predict(dushendux_test2,verbose=1)

In [ ]:
zui1 = zui*(zui>=0)
zuiattbilstmr1=r2_score(dushenduy_test2,zui1)
zuiattbilstmr1

第三版

In [70]:
filepath1 = "bilstmattention最好模型\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint1 = ModelCheckpoint(filepath=filepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num2_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a2=Dropout(0.2)(num2_bi_gru)
num2_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a2)
a2=Dropout(0.1)(num2_bi_gru)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru2=attention_bilstm(num2_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt2=Dense(1)(attbigru2)
attbilstm_model=Model(model_input,tt2)
attbilstm_model.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history = attbilstm_model.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint1)

In [ ]:
# 预测
attbilstm_pres2=attbilstm_model.predict(dushendux_test2,verbose=1)

In [ ]:
attbilstmy2_modify = attbilstm_pres2*(attbilstm_pres2>=0)
attbilstmr2=r2_score(dushenduy_test2,attbilstmy2_modify)
attbilstmr2

In [74]:
zuihaomodel2=load_model("bilstmattention最好模型\weights-improvement-657-0.00.hdf5")

In [ ]:
# 预测
zui2=zuihaomodel2.predict(dushendux_test2,verbose=1)

In [ ]:
zui3 = zui2*(zui2>=0)
zuiattbilstmr2=r2_score(dushenduy_test2,zui3)
zuiattbilstmr2

保存预测结果

In [ ]:
# 1.保存预测模型
attbilstm_model.save('论文2saveattbiLSTM_model')
pd.DataFrame(attbilstm_history.history).to_csv('论文2saveattbiLSTM_history.csv', index=False)

In [ ]:
 # 数据逆归一化
maxmin = [datal['PM2.5'].max(), datal['PM2.5'].min()]  # 原来的最大最小值，反放缩
print(maxmin)

In [ ]:
labels= np.array(dushenduy_test2)
labels.shape[1]

In [46]:
labels= np.array(dushenduy_test2)
for k in range(labels.shape[1]):
    labels[:, k] = labels[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
zuipreds = np.array(zui3)  # 转换为numpy形式
zuipreds.shape[1]

In [88]:
# 反归一化
zuipreds = np.array(zui3)  # 转换为numpy形式
zuipreds= zuipreds.reshape((-1,1))
zuifuture_len= zuipreds.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture_len):
    zuipreds[:, k] = zuipreds[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
zuipreds

In [ ]:
zuipreds.shape

In [95]:
zuipreds_modify=pd.DataFrame(zuipreds,columns=['y_pred'])
zuipreds_modify.to_csv('论文Bi-LSTM-Attention预测结果.csv',index=False)

In [ ]:
a1=pd.read_csv('论文Bi-LSTM-Attention预测结果.csv')
a1

In [99]:
a1=np.array(a1)

画图

In [ ]:
# 随机森林拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
# x = range(0,530,1)
plt.plot(labels[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(a1[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
plt.savefig("F:\jupyter\论文测试版本2\huangwenchao\北京昌平PM2.5预测\Bi-LSTM-Attention拟合图\论文Bi-LSTM-Attention拟合图530.jpg")

In [ ]:
# 随机森林拟合图2
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
x = range(530,1060,1)
plt.plot(x,labels[530:1060], "m-", label="real", alpha=0.9,ms=16)
plt.plot(x,a1[530:1060], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper right',fontsize=20)
# plt.tight_layout()
plt.savefig("F:\jupyter\论文测试版本2\huangwenchao\北京昌平PM2.5预测\Bi-LSTM-Attention拟合图\论文Bi-LSTM-Attention拟合图1060.jpg")

第四版

In [103]:
filepath2 = "bilstmattention最好模型第四版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint2 = ModelCheckpoint(filepath=filepath2,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num2_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a2=Dropout(0.2)(num2_bi_gru)
num2_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a2)
a3=Dropout(0.1)(num2_bi_gru2)
num2_bi_gru3=Bidirectional(LSTM(32,return_sequences=True))(a3)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru3=attention_bilstm(num2_bi_gru3)
# a=Dropout(0.2)(attbigru)
tt3=Dense(1)(attbigru3)
attbilstm_model3=Model(model_input,tt3)
attbilstm_model3.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history3 = attbilstm_model3.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint2)

In [105]:
zuihaomodel3=load_model("bilstmattention最好模型第四版\weights-improvement-692-0.00.hdf5")

In [ ]:
# 预测
zui3=zuihaomodel3.predict(dushendux_test2,verbose=1)

In [ ]:
zui4 = zui3*(zui3>=0)
zuiattbilstmr3=r2_score(dushenduy_test2,zui4)
zuiattbilstmr3

保存损失模型

In [ ]:
# 1.保存预测模型
attbilstm_model3.save('论文4saveattbiLSTM_model')
pd.DataFrame(attbilstm_history3.history).to_csv('论文4saveattbiLSTM_history.csv', index=False)

第5版

In [109]:
filepath3 = "bilstmattention最好模型第五版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint3 = ModelCheckpoint(filepath=filepath3,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num3_bi_gru=Bidirectional(LSTM(64,return_sequences=True))(model_input)
a3=Dropout(0.2)(num3_bi_gru)
num3_bi_gru2=Bidirectional(LSTM(32,return_sequences=True))(a3)
a4=Dropout(0.1)(num3_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru5=attention_bilstm(a4)
# a=Dropout(0.2)(attbigru)
tt5=Dense(1)(attbigru5)
attbilstm_model5=Model(model_input,tt5)
attbilstm_model5.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history5 = attbilstm_model5.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint3)

In [111]:
zuihaomodel5=load_model("bilstmattention最好模型第五版\weights-improvement-685-0.00.hdf5")

In [ ]:
# 预测
zui5=zuihaomodel5.predict(dushendux_test2,verbose=1)

In [ ]:
zui5_modify = zui5*(zui5>=0)
zuiattbilstmr5=r2_score(dushenduy_test2,zui5_modify)
zuiattbilstmr5

保存模型与预测值

In [ ]:
# 1.保存预测模型
attbilstm_model5.save('论文5saveattbiLSTM_model')
pd.DataFrame(attbilstm_history5.history).to_csv('论文5saveattbiLSTM_history.csv', index=False)

In [115]:
# 反归一化
zuipreds5 = np.array(zui5)  # 转换为numpy形式
zuipreds5= zuipreds5.reshape((-1,1))
zuifuture5_len= zuipreds5.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture5_len):
    zuipreds5[:, k] = zuipreds5[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [116]:
zuipreds5_modify=pd.DataFrame(zuipreds5,columns=['y_pred'])
zuipreds5_modify.to_csv('论文5Bi-LSTM-Attention预测结果.csv',index=False)

In [ ]:
r=pd.read_csv('论文5Bi-LSTM-Attention预测结果.csv')
r

In [ ]:
r2_score(labels,r)

第六版

In [121]:
filepath6 = "bilstmattention最好模型第六版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint6 = ModelCheckpoint(filepath=filepath6,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num_bi_gru6=Bidirectional(LSTM(64,return_sequences=True))(model_input)
num_bi_lstm6=Dropout(0.2)(num_bi_gru6)
num6_bi_gru2=Bidirectional(LSTM(32,return_sequences=True))(num_bi_lstm6)
a4=Dropout(0.1)(num6_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru5=attention_bilstm(a4)
aa=Dropout(0.1)(attbigru5)
tt6=Dense(1)(aa)
attbilstm_model6=Model(model_input,tt6)
attbilstm_model6.compile(optimizer='Adam', loss='mse')# metrics=['accuracy']
attbilstm_history6 = attbilstm_model6.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint6)

In [125]:
zuihaomodel6=load_model("bilstmattention最好模型第五版\weights-improvement-642-0.00.hdf5")

In [ ]:
# 预测
zui6=zuihaomodel6.predict(dushendux_test2,verbose=1)

In [ ]:
zui6_modify = zui6*(zui6>=0)
zuiattbilstmr6=r2_score(dushenduy_test2,zui6_modify)
zuiattbilstmr6

第七版

In [131]:
filepath7 = "bilstmattention最好模型第七版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint7 = ModelCheckpoint(filepath=filepath7,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num7_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a7=Dropout(0.2)(num7_bi_gru)
num7_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a7)
a77=Dropout(0.1)(num7_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru7=attention_bilstm(a77)
# a=Dropout(0.2)(attbigru)
tt7=Dense(1)(attbigru7)
attbilstm_model7=Model(model_input,tt7)
attbilstm_model7.compile(optimizer='Adam', loss='mae')# metrics=['accuracy']
attbilstm_history7 = attbilstm_model7.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint7)

In [134]:
zuihaomodel7=load_model("bilstmattention最好模型第七版\weights-improvement-968-0.01.hdf5")

In [ ]:
# 预测
zui7=zuihaomodel7.predict(dushendux_test2,verbose=1)

In [ ]:
zui7_modify = zui7*(zui7>=0)
zuiattbilstmr7=r2_score(dushenduy_test2,zui7_modify)
zuiattbilstmr7

第八版

In [137]:
filepath8 = "bilstmattention最好模型第八版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint8 = ModelCheckpoint(filepath=filepath8,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [64]:
from keras import optimizers

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num8_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a8=Dropout(0.2)(num8_bi_gru)
num8_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a8)
a88=Dropout(0.1)(num8_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru8=attention_bilstm(a88)
# a=Dropout(0.2)(attbigru)
tt8=Dense(1)(attbigru8)
attbilstm_model8=Model(model_input,tt8)
rmsp=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
attbilstm_model8.compile(optimizer=rmsp, loss='mse')# metrics=['accuracy']
attbilstm_history8 = attbilstm_model8.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint8)

In [144]:
zuihaomodel8=load_model("bilstmattention最好模型第八版\weights-improvement-471-0.00.hdf5")

In [ ]:
# 预测
zui8=zuihaomodel8.predict(dushendux_test2,verbose=1)

In [ ]:
zui8_modify = zui8*(zui8>=0)
zuiattbilstmr8=r2_score(dushenduy_test2,zui8_modify)
zuiattbilstmr8

第九版

In [147]:
filepath9 = "bilstmattention最好模型第九版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint9 = ModelCheckpoint(filepath=filepath9,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num9_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a9=Dropout(0.2)(num9_bi_gru)
num9_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a9)
a99=Dropout(0.2)(num9_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru9=attention_bilstm(a99)
# a=Dropout(0.2)(attbigru)
tt9=Dense(1)(attbigru9)
attbilstm_model9=Model(model_input,tt9)
ada=optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
attbilstm_model9.compile(optimizer=ada, loss='mse')# metrics=['accuracy']
attbilstm_history9 = attbilstm_model9.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint9)

In [155]:
zuihaomodel9=load_model("bilstmattention最好模型第九版\weights-improvement-793-0.00.hdf5")

In [ ]:
# 预测
zui9=zuihaomodel9.predict(dushendux_test2,verbose=1)

In [ ]:
zui9_modify = zui9*(zui9>=0)
zuiattbilstmr9=r2_score(dushenduy_test2,zui9_modify)
zuiattbilstmr9

第10版

In [158]:
filepath10 = "bilstmattention最好模型第十版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint10 = ModelCheckpoint(filepath=filepath10,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num10_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a10=Dropout(0.2)(num10_bi_gru)
num10_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a10)
a10=Dropout(0.2)(num10_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru10=attention_bilstm(a10)
# a=Dropout(0.2)(attbigru)
tt10=Dense(1)(attbigru10)
attbilstm_model10=Model(model_input,tt10)
sgd=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=True)
attbilstm_model10.compile(optimizer=sgd, loss='mse')# metrics=['accuracy']
attbilstm_history10 = attbilstm_model10.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint10)

In [160]:
zuihaomodel10=load_model("bilstmattention最好模型第十版\weights-improvement-992-0.00.hdf5")

In [ ]:
# 预测
zui10=zuihaomodel10.predict(dushendux_test2,verbose=1)

In [ ]:
zui10_modify = zui10*(zui10>=0)
zuiattbilstmr10=r2_score(dushenduy_test2,zui10_modify)
zuiattbilstmr10

第11版

In [185]:
# 添加注意力机制
def attention_bilstm11(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(16, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [165]:
filepath11= "bilstmattention最好模型第十一版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint11 = ModelCheckpoint(filepath=filepath11,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a11=Dropout(0.5)(num11_bi_gru)
num11_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a11)
a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru11=attention_bilstm11(a111)
# a=Dropout(0.2)(attbigru)
tt11=Dense(1)(attbigru11)
attbilstm_model11=Model(model_input,tt11)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model11.compile(optimizer=adaa, loss='mse')# metrics=['accuracy']
attbilstm_history11 = attbilstm_model11.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint11)

In [167]:
zuihaomodel11=load_model("bilstmattention最好模型第十一版\weights-improvement-683-0.00.hdf5")

In [ ]:
# 预测
zui11=zuihaomodel11.predict(dushendux_test2,verbose=1)

In [ ]:
zui11_modify = zui11*(zui11>=0)
zuiattbilstmr11=r2_score(dushenduy_test2,zui11_modify)
zuiattbilstmr11

第12版

In [170]:
filepath12= "bilstmattention最好模型第十二版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint12 = ModelCheckpoint(filepath=filepath12,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [178]:
from keras import regularizers

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num12_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a12=Dropout(0.5)(num12_bi_gru)
num12_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a12)
a112=Dropout(0.2)(num12_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru12=attention_bilstm11(a112)
# a=Dropout(0.2)(attbigru)
tt12=Dense(1,kernel_regularizer=regularizers.l2(0.01))(attbigru12)
attbilstm_model12=Model(model_input,tt12)
rmsp=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)
attbilstm_model12.compile(optimizer=rmsp, loss='mse',metrics=['mse'])# metrics=['accuracy']
attbilstm_history12 = attbilstm_model12.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint12)

In [181]:
zuihaomodel12=load_model("bilstmattention最好模型第十二版\weights-improvement-1000-0.01.hdf5")

In [ ]:
# 预测
zui12=zuihaomodel12.predict(dushendux_test2,verbose=1)

In [ ]:
zui12_modify = zui12*(zui12>=0)
zuiattbilstmr12=r2_score(dushenduy_test2,zui12_modify)
zuiattbilstmr12

第13版

In [186]:
filepath13= "bilstmattention最好模型第十三版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint13 = ModelCheckpoint(filepath=filepath13,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num13_bi_gru=Bidirectional(LSTM(128,return_sequences=True))(model_input)
a13=Dropout(0.5)(num13_bi_gru)
num13_bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(a13)
a113=Dropout(0.2)(num13_bi_gru2)
b13=Bidirectional(LSTM(32,return_sequences=True))(a113)
b113=Dropout(0.2)(b13)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru13=attention_bilstm11(b113)
# a=Dropout(0.2)(attbigru)
tt13=Dense(1)(attbigru13)
attbilstm_model13=Model(model_input,tt13)
rmsp=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.1)
attbilstm_model13.compile(optimizer=rmsp, loss='mse',metrics=['mse'])# metrics=['accuracy']
attbilstm_history13= attbilstm_model13.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=checkpoint13)

In [188]:
zuihaomodel13=load_model("bilstmattention最好模型第十三版\weights-improvement-1000-0.00.hdf5")

In [ ]:
# 预测
zui13=zuihaomodel13.predict(dushendux_test2,verbose=1)

In [ ]:
zui13_modify = zui13*(zui13>=0)
zuiattbilstmr13=r2_score(dushenduy_test2,zui13_modify)
zuiattbilstmr13

In [192]:
filepath14= "bilstmattention最好模型第十四版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint14 = ModelCheckpoint(filepath=filepath14,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru14=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num14_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru14)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru14=attention_bilstm11(num14_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt14=Dense(1)(attbigru14)
attbilstm_model14=Model(model_input,tt14)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model14.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history14 = attbilstm_model14.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint14)

In [195]:
zuihaomodel14=load_model("bilstmattention最好模型第十四版\weights-improvement-714-0.00.hdf5")

In [ ]:
# 预测
zui14=zuihaomodel14.predict(dushendux_test2,verbose=1)

In [ ]:
zui14_modify = zui14*(zui14>=0)
zuiattbilstmr14=r2_score(dushenduy_test2,zui14_modify)
zuiattbilstmr14

第15版本

In [60]:
filepath15= "bilstmattention最好模型第十五版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint15 = ModelCheckpoint(filepath=filepath15,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
con1=Conv1D(32,3,activation='relu',input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(model_input)
# max1=MaxPooling2D((2,2)))(con1)
max1 = MaxPooling1D(pool_size = 2, name = 'max_pool_1')(con1)
# flat1=Flatten()(max1)
bi_lstm_att1=Bidirectional(LSTM(128,return_sequences=True))(max1)
t1=Dropout(0.5)(bi_lstm_att1)
bi_gru2=Bidirectional(LSTM(64,return_sequences=True))(t1)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
t11=Dropout(0.3)(bi_gru2)
attbigru=attention_bilstm(t11)
# a=Dropout(0.2)(attbigru)
newattbilstm_model=Model(model_input,attbigru)
newattbilstm_model.compile(optimizer='Adam', loss='mse',metrics='mae')# metrics=['accuracy']
newattbilstm_model = newattbilstm_model.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint15)

第16版本

In [81]:
filepath16= "bilstmattention最好模型第十六版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint16 = ModelCheckpoint(filepath=filepath16,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru16=Bidirectional(GRU(128,dropout=0.2,return_sequences=True))(model_input)
a16=Dropout(0.5)(num11_bi_gru16)
num16_bi_gru2=Bidirectional(GRU(64,dropout=0.2,return_sequences=True))(a16)
a116=Dropout(0.2)(num16_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru16=attention_bilstm(a116)
# a=Dropout(0.2)(attbigru)
tt16=Dense(1)(attbigru16)
attbilstm_model16=Model(model_input,tt16)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model16.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history16 = attbilstm_model16.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint16)

In [88]:
zuihaomodel16=load_model("bilstmattention最好模型第十六版\weights-improvement-748-0.00.hdf5")

In [ ]:
# 预测
zui16=zuihaomodel16.predict(dushendux_test2,verbose=1)

In [ ]:
zui16_modify = zui16*(zui16>=0)
zuiattbilstmr16=r2_score(dushenduy_test2,zui16_modify)
zuiattbilstmr16

保存模型

In [ ]:
# 1.保存预测模型
attbilstm_model16.save('论文0.932saveattbiLSTM_model')
pd.DataFrame(attbilstm_history16.history).to_csv('论文0.932saveattbiLSTM_history.csv', index=False)

In [110]:
# 反归一化
zuipreds16 = np.array(zui16_modify)  # 转换为numpy形式
zuipreds16= zuipreds16.reshape((-1,1))
zuifuture16_len= zuipreds16.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture16_len):
    zuipreds16[:, k] = zuipreds16[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [111]:
zuipreds16_modify=pd.DataFrame(zuipreds16,columns=['y_pred'])
zuipreds16_modify.to_csv('论文0.932Bi-LSTM-Attention预测结果.csv',index=False)

In [ ]:
r2_score(label,pd.read_csv('论文0.932Bi-LSTM-Attention预测结果.csv'))

In [114]:
q=pd.read_csv('论文0.932Bi-LSTM-Attention预测结果.csv')

In [ ]:
# 计算rmse
np.sqrt(mean_squared_error(label,q))

In [117]:
q=np.array(q)

In [118]:
p=pd.read_csv('论文0.932saveattbiLSTM_history.csv')

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(p['val_loss'],label='Bi-LSTM-Attention_loss_value')
plt.plot(p['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.932Bi-LSTMAttention损失曲线.jpg")
plt.show()

第17版本

In [83]:
filepath17= "bilstmattention最好模型第十七版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint17 = ModelCheckpoint(filepath=filepath17,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru17=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(model_input)
# a16=Dropout(0.5)(num11_bi_gru16)
num17_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru17)
# a116=Dropout(0.2)(num16_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru17=attention_bilstm(num17_bi_gru2)
a17=Dropout(0.2)(attbigru17)
tt17=Dense(1)(a17)
attbilstm_model17=Model(model_input,tt17)
adaa=optimizers.Adam(lr=0.0001)
attbilstm_model17.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history17 = attbilstm_model17.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=checkpoint17)

In [91]:
zuihaomodel17=load_model("bilstmattention最好模型第十七版\weights-improvement-737-0.00.hdf5")

In [ ]:
# 预测
zui17=zuihaomodel17.predict(dushendux_test2,verbose=1)

In [ ]:
zui17_modify = zui17*(zui17>=0)
zuiattbilstmr17=r2_score(dushenduy_test2,zui17_modify)
zuiattbilstmr17

第18版

In [130]:
filepath18="bilstmattention最好模型第十八版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list18=[
    ModelCheckpoint(filepath=filepath18,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
    ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru18=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(model_input)
a18=Dropout(0.5)(num11_bi_gru18)
num18_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(a18)
a118=Dropout(0.2)(num18_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru18=attention_bilstm(a118)
# a=Dropout(0.2)(attbigru)
tt18=Dense(1)(attbigru18)
attbilstm_model18=Model(model_input,tt18)
adaa=optimizers.Adam(learning_rate=0.001)
attbilstm_model18.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history18 = attbilstm_model18.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=callback_list18)

In [134]:
zuihaomodel18=load_model("bilstmattention最好模型第十八版\weights-improvement-30-0.00.hdf5")

In [ ]:
# 预测
zui18=zuihaomodel18.predict(dushendux_test2,verbose=1)

In [ ]:
zui18_modify = zui18*(zui18>=0)
zuiattbilstmr18=r2_score(dushenduy_test2,zui18_modify)
zuiattbilstmr18

In [ ]:
第19版本

In [132]:
filepath19="bilstmattention最好模型第十九版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list19=[
    ModelCheckpoint(filepath=filepath19,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
#     ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru19=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(model_input)
a19=Dropout(0.5)(num11_bi_gru19)
num19_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(a19)
a119=Dropout(0.2)(num19_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru19=attention_bilstm(a119)
# a=Dropout(0.2)(attbigru)
tt19=Dense(1)(attbigru19)
attbilstm_model19=Model(model_input,tt19)
adaa=optimizers.Adam(learning_rate=0.0001)
attbilstm_model19.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history19 = attbilstm_model19.fit(train_data_single, validation_data=val_data_single,epochs=800, verbose=1,callbacks=callback_list19)

In [137]:
zuihaomodel19=load_model("bilstmattention最好模型第十九版\weights-improvement-772-0.00.hdf5")

In [ ]:
# 预测
zui19=zuihaomodel19.predict(dushendux_test2,verbose=1)

In [ ]:
zui19_modify = zui19*(zui19>=0)
zuiattbilstmr19=r2_score(dushenduy_test2,zui19_modify)
zuiattbilstmr19

第20版

In [140]:
filepath20="bilstmattention最好模型第二十版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list20=[
    ModelCheckpoint(filepath=filepath20,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
#     ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru20=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(model_input)
# a19=Dropout(0.5)(num11_bi_gru19)
num20_bi_gru2=Bidirectional(LSTM(64,dropout=0.5,return_sequences=True))(num11_bi_gru20)
# a119=Dropout(0.2)(num19_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru20=attention_bilstm(num20_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt20=Dense(1)(attbigru20)
attbilstm_model20=Model(model_input,tt20)
adaa=optimizers.Adam(learning_rate=0.002)
attbilstm_model20.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history20 = attbilstm_model20.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=callback_list20)

第21版

In [142]:
filepath21="bilstmattention最好模型第二十一版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list21=[
    ModelCheckpoint(filepath=filepath21,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
#     ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru21=Bidirectional(LSTM(128,dropout=0.3,return_sequences=True))(model_input)
# a19=Dropout(0.5)(num11_bi_gru19)
num21_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru21)
# a119=Dropout(0.2)(num19_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru21=attention_bilstm(num21_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt21=Dense(1)(attbigru21)
attbilstm_model21=Model(model_input,tt21)
adaa=optimizers.Adam(learning_rate=0.002)
attbilstm_model21.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history21 = attbilstm_model21.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=callback_list21)

保存模型

In [145]:
zuihaomodel21=load_model("bilstmattention最好模型第二十一版\weights-improvement-661-0.00.hdf5")

In [ ]:
# 预测
zui21=zuihaomodel21.predict(dushendux_test2,verbose=1)

In [ ]:
zui21_modify = zui21*(zui21>=0)
zuiattbilstmr21=r2_score(dushenduy_test2,zui21_modify)
zuiattbilstmr21

In [ ]:
# 1.保存预测模型
attbilstm_model21.save('论文0.926saveattbiLSTM_model')
pd.DataFrame(attbilstm_history21.history).to_csv('论文0.926saveattbiLSTM_history.csv', index=False)

In [149]:
# 反归一化
zuipreds21= np.array(zui21_modify)  # 转换为numpy形式
zuipreds21= zuipreds16.reshape((-1,1))
zuifuture21_len= zuipreds21.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture21_len):
    zuipreds21[:, k] = zuipreds21[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [150]:
zuipreds21_modify=pd.DataFrame(zuipreds21,columns=['y_pred'])
zuipreds21_modify.to_csv('论文0.926Bi-LSTM-Attention预测结果.csv',index=False)

In [ ]:
p21=pd.read_csv('论文0.926saveattbiLSTM_history.csv')
p21

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(p21['val_loss'],label='Bi-LSTM-Attention_loss_value')
plt.plot(p21['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.926Bi-LSTMAttention损失曲线.jpg")
plt.show()

第22版本

In [161]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [162]:
filepath22="bilstmattention最好模型第二十二版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list22=[
    ModelCheckpoint(filepath=filepath22,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
#     ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru22=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(model_input)
# a19=Dropout(0.5)(num11_bi_gru19)
num22_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru22)
# a119=Dropout(0.2)(num19_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru22=attention_bilstm(num22_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt22=Dense(128)(attbigru22)
tt222=Dense(1)(tt22)
attbilstm_model22=Model(model_input,tt222)
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=100,
    decay_rate=0.9)
adaa=optimizers.Adam(learning_rate=lr_schedule)
attbilstm_model22.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history22 = attbilstm_model22.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=callback_list22)

第23版本

In [164]:
filepath23="bilstmattention最好模型第二十三版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback_list23=[
    ModelCheckpoint(filepath=filepath23,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1),
#     ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)
]

In [59]:
# 添加注意力机制
def attention_bilstm64(inputs):
    """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
    hidden_states = inputs
    hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
    attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
    pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
    attention_vector = Dense(64, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return attention_vector

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru23=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num123_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru23)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru23=attention_bilstm64(num123_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt23=Dense(1)(attbigru23)
attbilstm_model23=Model(model_input,tt23)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model23.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history23 = attbilstm_model23.fit(train_data_single, validation_data=val_data_single,epochs=1000, verbose=1,callbacks=callback_list23)

In [168]:
zuihaomodel23=load_model("bilstmattention最好模型第二十三版\weights-improvement-983-0.00.hdf5")

In [ ]:
# 预测
zui23=zuihaomodel23.predict(dushendux_test2,verbose=1)

In [ ]:
zui23_modify = zui23*(zui23>=0)
zuiattbilstmr23=r2_score(dushenduy_test2,zui23_modify)
zuiattbilstmr23

保存模型

In [ ]:
# 1.保存预测模型
attbilstm_model23.save('论文0.934saveattbiLSTM_model')
pd.DataFrame(attbilstm_history23.history).to_csv('论文0.934saveattbiLSTM_history.csv', index=False)

In [172]:
# 反归一化
zuipreds23= np.array(zui23_modify)  # 转换为numpy形式
zuipreds23= zuipreds23.reshape((-1,1))
zuifuture23_len= zuipreds23.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture23_len):
    zuipreds23[:, k] = zuipreds23[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
p23=pd.read_csv('论文0.934saveattbiLSTM_history.csv')
p23

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(p23['val_loss'],label='Bi-LSTM-Attention_loss_value')
# plt.plot(p23['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.934Bi-LSTMAttention损失曲线.jpg")
plt.show()

第24版本

In [33]:
filepath24="bilstmattention最好模型第二十四版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback24=ModelCheckpoint(filepath=filepath24,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru24=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num124_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru24)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru24=attention_bilstm64(num124_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt24=Dense(1)(attbigru24)
attbilstm_model24=Model(model_input,tt24)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model24.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history24 = attbilstm_model24.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=callback24)

第25版

In [20]:
filepath25="bilstmattention最好模型第二十五版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback25=ModelCheckpoint(filepath=filepath25,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru25=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num125_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru25)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru25=attention_bilstm64(num125_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt25=Dense(1)(attbigru25)
attbilstm_model25=Model(model_input,tt25)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model25.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history25 = attbilstm_model25.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=callback25)

In [38]:
zuihaomodel25=load_model("bilstmattention最好模型第二十五版\weights-improvement-1422-0.00.hdf5")

In [ ]:
# 预测
zui25=zuihaomodel25.predict(dushendux_test2,verbose=1)

In [ ]:
zui25_modify = zui25*(zui25>=0)
zuiattbilstmr25=r2_score(dushenduy_test2,zui25_modify)
zuiattbilstmr25

保存模型

In [ ]:
# 1.保存预测模型
attbilstm_model25.save('论文0.937saveattbiLSTM_model')
pd.DataFrame(attbilstm_history25.history).to_csv('论文0.937saveattbiLSTM_history.csv', index=False)

In [45]:
# 反归一化
zuipreds25= np.array(zui25_modify)  # 转换为numpy形式
zuipreds25= zuipreds25.reshape((-1,1))
zuifuture25_len= zuipreds25.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture25_len):
    zuipreds25[:, k] = zuipreds25[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [46]:
zuipreds25_modify=pd.DataFrame(zuipreds25,columns=['y_pred'])
zuipreds25_modify.to_csv('论文0.937Bi-LSTM-Attention预测结果.csv',index=False)

读取模型

In [38]:
zuihaomodel25=load_model("bilstmattention最好模型第二十五版\weights-improvement-1422-0.00.hdf5")

In [ ]:
# 预测
zui25=zuihaomodel25.predict(dushendux_test2,verbose=1)

In [ ]:
zui25_modify = zui25*(zui25>=0)
zuiattbilstmr25=r2_score(dushenduy_test2,zui25_modify)
zuiattbilstmr25

In [ ]:
zui25_modify

In [57]:
# 反归一化
zuipreds251= np.array(zui25_modify)  # 转换为numpy形式
zuipreds251= zuipreds251.reshape((-1,1))
zuifuture251_len= zuipreds251.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(zuifuture251_len):
    zuipreds251[:, k] = zuipreds251[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [ ]:
zuipreds251

In [59]:
zuipreds25_modify=pd.DataFrame(zuipreds251,columns=['y_pred'])
zuipreds25_modify.to_csv('论文0.937Bi-LSTM-Attention预测结果.csv',index=False)

In [60]:
dumoxing25=pd.read_csv('论文0.937Bi-LSTM-Attention预测结果.csv')

In [ ]:
r2_score(labels,dumoxing25)

In [43]:
p25=pd.read_csv('论文0.937saveattbiLSTM_history.csv')

In [45]:
p25=np.array(p25)

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(p25['val_loss'],label='Bi-LSTM-Attention_loss_value')
# plt.plot(p23['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('Bi-LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.937Bi-LSTMAttention损失曲线.jpg")
plt.show()

In [ ]:
# 注意力机制拟合图1
plt.figure(figsize=(25, 8))
ax = plt.subplot()
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False
plt.grid(color="k", linestyle=":")
ax.set_title('Bi-LSTM-Attention模型',fontsize=25)
# x = range(0,530,1)
plt.plot(labels[0:530], "m-", label="real", alpha=0.9,ms=16)
plt.plot(dumoxing25[0:530], "c-.", label="pred", alpha=0.9,ms=16)
plt.xlabel('time/h',fontsize=25)
plt.ylabel('PM2.5/(ug/m^3)',fontsize=25)
# plt.rcParams.update({'font.size': 20})
plt.legend(loc='upper left',fontsize=20)
plt.savefig("F:\jupyter\论文测试版本2\huangwenchao\北京昌平PM2.5预测\注意力机制预测图\论文注意力机制拟合图530.jpg")

In [34]:
filepath26="bilstmattention最好模型第二十六版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback26=ModelCheckpoint(filepath=filepath26,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru26=Bidirectional(LSTM(128,dropout=0.3,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num126_bi_gru2=Bidirectional(LSTM(64,dropout=0.3,return_sequences=True))(num11_bi_gru26)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
attbigru26=attention_bilstm(num126_bi_gru2)
# a=Dropout(0.2)(attbigru)
tt26=Dense(1)(attbigru26)
attbilstm_model26=Model(model_input,tt26)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model26.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history26 = attbilstm_model26.fit(train_data_single100, validation_data=val_data_single100,epochs=1500, verbose=1,callbacks=callback26)

# bi-lstm

In [32]:
bfilepath1="bilstm模型第一版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
bcallback1=ModelCheckpoint(filepath=bfilepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num1_bi_lstm1=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
# a11=Dropout(0.5)(num11_bi_gru)
num2_bi_lstm1=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num1_bi_lstm1)
# a111=Dropout(0.2)(num11_bi_gru2)
# attbilstm_model.add(Bidirectional(GRU(64), input_shape=trainx.shape[-2:],return_sequences=True))
# a=Dropout(0.2)(attbigru)
tt2=Dense(1)(num2_bi_lstm1)
bilstm_model1=Model(model_input,tt2)
adaa=optimizers.Adam(lr=0.001)
bilstm_model1.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
bilstm_history1 = bilstm_model1.fit(train_data_single100, validation_data=val_data_single100,epochs=1500, verbose=1,callbacks=bcallback1)

保存模型

In [ ]:
# 预测
bilstm1=bilstm_model1.predict(dushendux_test2,verbose=1)

In [ ]:
bilstm1.shape

In [40]:
bilstm1=bilstm1.reshape(-1,1)

第二版

In [43]:
bfilepath2="bilstm模型第二版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
bcallback2=ModelCheckpoint(filepath=bfilepath2,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
bilstm_model2 = Sequential()
bilstm_model2.add(Bidirectional(LSTM(128,dropout=0.2,input_shape=dushendux_train2.shape[-2:],return_sequences=True)))
bilstm_model2.add(Bidirectional(LSTM(64,dropout=0.2,input_shape=dushendux_train2.shape[-2:])))
bilstm_model2.add(Dense(1))
adaa=optimizers.Adam(lr=0.001)
bilstm_model2.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
bilstm_history2 = bilstm_model2.fit(train_data_single100, epochs=1500,validation_data=val_data_single100,verbose=1,callbacks=bcallback2)

第27版

In [34]:
filepath27="bilstmattention最好模型第二十七版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback27=ModelCheckpoint(filepath=filepath27,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru27=Bidirectional(LSTM(128,dropout=0.2,return_sequences=True))(model_input)
num127_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru27)
attbigru27=attention_bilstm(num127_bi_gru2)
tt27=Dense(1)(attbigru27)
attbilstm_model27=Model(model_input,tt27)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model27.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history27 = attbilstm_model27.fit(train_data_single100, validation_data=val_data_single100,epochs=1000, verbose=1,callbacks=callback27)

In [56]:
cnnfilepath1="cnn-bilstmattention第1版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
cnncallback1=ModelCheckpoint(filepath=cnnfilepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造cnn-bilstm-att
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
con1=Conv1D(16,1,input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(model_input)
max1 = MaxPooling1D(pool_size = 1)(con1)
# con2=Conv1D(16,1,input_shape=(dushendux_train2.shape[-2],dushendux_train2.shape[-1]))(max1)
cnn_bilstm_att1=Bidirectional(LSTM(128,dropout=0.5,return_sequences=True))(max1)
# cnn_bilstm2_att1=Bidirectional(LSTM(64,dropout=0.3,return_sequences=True))(cnn_bilstm_att1)
attbigru1=attention_bilstm32(cnn_bilstm_att1)
den1=Dense(1)(attbigru1)
cnnattbilstm_model1=Model(model_input,den1)
adaa=optimizers.Adam(lr=0.001)
cnnattbilstm_model1.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
cnnattbilstm_model_history = cnnattbilstm_model1.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=cnncallback1)

# LSTM-Attention

lstmattention第一版

In [33]:
lstmfilepath1="lstmattention第1版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
lstmcallback1=ModelCheckpoint(filepath=lstmfilepath1,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num1_lstm1=LSTM(128,dropout=0.2,return_sequences=True)(model_input)
num2_lstm1=LSTM(64,dropout=0.2,return_sequences=True)(num1_lstm1)
attlstm1=attention_bilstm(num2_lstm1)
# tlstm1=Dense(1)(attlstm1)
tlstm1=Dense(1)(attlstm1)
attlstm_model1=Model(model_input,tlstm1)
adaa=optimizers.Adam(lr=0.001)
attlstm_model1.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attlstm_history1 = attlstm_model1.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=lstmcallback1)

In [37]:
attlstm_zuihaomodel1=load_model("lstmattention第1版\weights-improvement-709-0.00.hdf5")

In [ ]:
# 预测
attlstm1=attlstm_zuihaomodel1.predict(dushendux_test2,verbose=1)

In [ ]:
attlstm1_modify = attlstm1*(attlstm1>=0)
zuiattlstmr1=r2_score(dushenduy_test2,attlstm1_modify)
zuiattlstmr1

保存模型及预测结果

In [ ]:
# 1.保存预测模型
attlstm_model1.save('论文0.939saveattLSTM_model')
pd.DataFrame(attlstm_history1.history).to_csv('论文0.939saveattLSTM_history.csv', index=False)

In [40]:
# 反归一化
attlstm_zuipreds1= np.array(attlstm1_modify)  # 转换为numpy形式
attlstm_zuipreds1= attlstm_zuipreds1.reshape((-1,1))
attlstm_zuipreds1_len= attlstm_zuipreds1.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(attlstm_zuipreds1_len):
    attlstm_zuipreds1[:, k] = attlstm_zuipreds1[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [42]:
attlstm_zuipreds1_modify=pd.DataFrame(attlstm_zuipreds1,columns=['y_pred'])
attlstm_zuipreds1_modify.to_csv('论文0.939LSTM-Attention预测结果.csv',index=False)

In [43]:
a10=pd.read_csv('论文0.939LSTM-Attention预测结果.csv')

In [ ]:
r2_score(labels,a10)

In [52]:
q1=pd.read_csv('论文0.939saveattLSTM_history.csv')
# q1=np.array(q1)

In [ ]:
q1

In [ ]:
plt.figure(figsize=(12,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(q1['val_loss'],label='LSTM-Attention_loss_value')
# plt.plot(p23['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.939LSTMAttention损失曲线.jpg")
plt.show()

第2版

In [38]:
lstmfilepath2="lstmattention第2版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
lstmcallback2=ModelCheckpoint(filepath=lstmfilepath2,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num1_lstm2=LSTM(128,dropout=0.3,return_sequences=True)(model_input)
num2_lstm2=LSTM(64,dropout=0.3,return_sequences=True)(num1_lstm2)
attlstm2=attention_bilstm32(num2_lstm2)
# tlstm1=Dense(1)(attlstm1)
tlstm2=Dense(1)(attlstm2)
attlstm_model2=Model(model_input,tlstm2)
adaa=optimizers.Adam(lr=0.0005)
attlstm_model2.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attlstm_history2 = attlstm_model2.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=lstmcallback2)

In [40]:
attlstm_zuihaomodel2=load_model("lstmattention第2版\weights-improvement-1023-0.00.hdf5")

In [ ]:
# 预测
attlstm2=attlstm_zuihaomodel2.predict(dushendux_test2,verbose=1)

In [ ]:
attlstm2_modify = attlstm2*(attlstm2>=0)
zuiattlstmr2=r2_score(dushenduy_test2,attlstm2_modify)
zuiattlstmr2

In [ ]:
# 1.保存预测模型
attlstm_model2.save('论文0.943saveattLSTM_model')
pd.DataFrame(attlstm_history2.history).to_csv('论文0.943saveattLSTM_history.csv', index=False)

In [44]:
# 反归一化
attlstm_zuipreds2= np.array(attlstm2_modify)  # 转换为numpy形式
attlstm_zuipreds2= attlstm_zuipreds2.reshape((-1,1))
attlstm_zuipreds2_len= attlstm_zuipreds2.shape[1]  # 获取要预测的天数长度，8预测5的话那就是5
for k in range(attlstm_zuipreds2_len):
    attlstm_zuipreds2[:, k] = attlstm_zuipreds2[:, k] * (maxmin[0] - maxmin[1]) + maxmin[1]  # 反归一化

In [45]:
attlstm_zuipreds2_modify=pd.DataFrame(attlstm_zuipreds2,columns=['y_pred'])
attlstm_zuipreds2_modify.to_csv('论文0.943LSTM-Attention预测结果.csv',index=False)

In [46]:
a11=pd.read_csv('论文0.943LSTM-Attention预测结果.csv')

In [ ]:
r2_score(labels,a11)

In [48]:
q2=pd.read_csv('论文0.943saveattLSTM_history.csv')

In [ ]:
plt.figure(figsize=(10,6))
plt.rcParams['font.sans-serif']=['Fangsong']
plt.rcParams['axes.unicode_minus']=False
# plt.plot(att_model_history3.history['loss'],label=['train_loss'])
plt.grid(color="k", linestyle=":")
plt.plot(q2['val_loss'],label='LSTM-Attention_loss_value')
# plt.plot(p23['loss'],label='Bi-LSTM-Attention_loss_value')
plt.xlabel('time/h',fontsize=25)
plt.ylabel('LSTM-Attention_loss_value',fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.rcParams.update({'font.size': 14})
plt.legend(fontsize=20)
plt.savefig("论文0.943LSTMAttention损失曲线.jpg")
plt.show()

结合xgboost结果

In [ ]:
xgbpreds_cha=np.absolute(labels-xgbpreds)
xgbpreds_cha

In [ ]:
attlstm_cha=np.absolute(labels-attlstm_zuipreds2)
attlstm_cha

In [ ]:
attlstm_zuipreds2

In [ ]:
xgbpreds

In [ ]:
xgbpreds_cha/(xgbpreds_cha+attlstm_cha)

In [127]:
cha1=(xgbpreds_cha/(xgbpreds_cha+attlstm_cha))*attlstm_zuipreds2
cha2=(attlstm_cha/(xgbpreds_cha+attlstm_cha))*xgbpreds

In [ ]:
cha1

In [ ]:
cha2

In [ ]:
zui_jiehe=cha1+cha2
zui_jiehe[:50]

In [ ]:
zui_jiehe

In [ ]:
attlstm_zuipreds2[:50]

In [ ]:
labels

In [ ]:
r2_score(labels,attlstm_zuipreds2)

In [ ]:
r2_score(labels,np.absolute(zui_jiehe))

In [137]:
zui_jiehe_data=pd.DataFrame(zui_jiehe,columns=['y_pred'])
zui_jiehe_data.to_csv('论文0.943att-lstm与xgboost结合=0.968.csv',index=False)

第3版

In [42]:
lstmfilepath3="lstmattention第3版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
lstmcallback3=ModelCheckpoint(filepath=lstmfilepath3,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num1_lstm3=LSTM(128,dropout=0.3,return_sequences=True)(model_input)
num3_lstm2=LSTM(64,dropout=0.2,return_sequences=True)(num1_lstm3)
attlstm3=attention_bilstm32(num3_lstm2)
# tlstm1=Dense(1)(attlstm1)
tlstm3=Dense(1)(attlstm3)
attlstm_model3=Model(model_input,tlstm3)
adaa=optimizers.Adam(lr=0.001)
attlstm_model3.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attlstm_history3 = attlstm_model3.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=lstmcallback3)

第28版

In [38]:
filepath28="bilstmattention最好模型第二十八版\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
callback28=ModelCheckpoint(filepath=filepath28,monitor='val_loss',mode='auto' ,save_best_only='True',verbose = 1)

In [ ]:
# 构造BI_GRU
# attbilstm_model = Sequential()
time_steps=120
input_dim=8
model_input=Input(shape=(time_steps,input_dim))
num11_bi_gru28=LSTM(128,dropout=0.3,return_sequences=True)(model_input)
# num127_bi_gru2=Bidirectional(LSTM(64,dropout=0.2,return_sequences=True))(num11_bi_gru27)
attbigru28=attention_bilstm(num11_bi_gru28)
tt28=Dense(1)(attbigru28)
attbilstm_model28=Model(model_input,tt28)
adaa=optimizers.Adam(lr=0.001)
attbilstm_model28.compile(optimizer=adaa, loss='mse',metrics='mae')# metrics=['accuracy']
attbilstm_history28 = attbilstm_model28.fit(train_data_single, validation_data=val_data_single,epochs=1500, verbose=1,callbacks=callback28)